<a href="https://colab.research.google.com/github/BeeGassy/Deep_Q-Learning/blob/main/DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms as T
import numpy as np
import matplotlib.pyplot as plt
import gym

In [2]:
class Model(nn.Module):
    #takes the # of frames stacked and the possible outputs (move right, left, etc)
    def __init__(self, numberStacked, possibleOutputs):
        hiddenKernels = 16
        
        #sizePostConvolution = #figure this out, depends on how we modify the env
        self.conv1 = nn.Conv2d(numberStacked, hiddenKernels, 2)
        self.rl = nn.ReLU()
        self.conv2 = nn.Conv2d(hiddenKernels, hiddenKernels, 2)
        self.fc1 = nn.Linear(sizePostConvolution, possibleOutputs)

    def forward(self, stackedState):
        x = self.conv1(stackedState)
        x = self.rl(x)
        x = self.conv2(stackedState)
        print(x.size())
        x = torch.view(-1, x.size()[1] * x.size()[2] * x.size()[3])
        x = self.fc1(x)
        return x

In [3]:
class Agent():
  replay_buffer_list = []

  def __init__(self, size, numberStacked, possibleOutputs):
    self.rb_size = size
    self.n = Model(4, len(DQN.test_env.action_space))

  def action_value(input_images):
    output = self.n(input_images)
    print(output)
    action = torch.argmax(output)
    return action

  def sample_replay_buffer(self, batch_size):
    perm = torch.tensor(self.replay_buffer_list)
    idx = perm[:batch_size]
    samples = tensor[idx]
    return samples

  def SGD(input_y_j, input_mini_batch):
    current_reward = torch.tensor(input_y_j, requires_grad=True)
    size = len(input_mini_batch)
    for batch in input_mini_batch:
      state, action, observation, reward, done = batch

      loss = torch.nn.MSELoss(self.n(observation), current_reward)
      loss_prime = loss.backward()

    return loss
        
    
  

   

In [4]:
class DQN():
  DEFAULT_ENV_NAME = 'PongNoFrameskip-v4'
  test_env = gym.make(DEFAULT_ENV_NAME)
  episodes = 400
  time_in_episode = 1000000
  epsilon = 0.5
  batch_size = 32

  def DQN(self, test_env, episodes, time_in_episode, epsilon):
    self.test_env = test_env
    self.episodes = episodes
    self.time_in_episode = time_in_episode
    self.epsilon = epsilon

  def emulator(self):
    state = self.test_env.reset()
    action = self.test_env.action_space.sample()
    observation, reward, _, _ = self.test_env.step(action)
    transition = (state, action, observation, reward)

    return transition

  def execute_action(self, input_action):
    observation, reward, done, _ = self.test_env.step(input_action)
    transition = (state, action, observation, reward, done)
    Agent.replay_buffer_list.insert(state, action, observation, reward)

    return transition

  def preprocessing(self, input_observation):
    np_observation = np.transpose(observation, (0, 3, 1, 2))
    copy_observation = np_observation.copy()
    torch_observation = torch.tensor(copy_observation, dtype=torch.float)
    transform = T.Grayscale()
    grey_scaled_observation = transform(torch_observation)

    return grey_scaled_observation

  def stack_frames(self, input_env, stack_count):
    enviroment = gym.FrameStack(input_env, stack_count)

    return enviroment
    
  
  def dqn(self):
    re_buff = Agent.replay_buffer()
    rewardVal = -1

    for e in range(episodes):
      rewardVal = 0
      
      #initialize episode and get first transition
      initial_transition = emulator()
      state, action, observation, reward, _ = initial_transition

      #preprocess data
      self.test_env = stack_frames(self.test_env, 4)
      grey_scaled_observation = preprocessing(observation)

      for time_step in range(time_in_episode):
        random_action_prob = random.uniform(0.0, 1.0)
        if random_action_prob < epsilon:
          action()
        else: 
          action_at_time_step = action_value(grey_scaled_observation)
          transition = execute_action(action_at_time_step)
          state, action, observation, reward, done = transition
          grey_scaled_observation = preprocessing(observation)
          re_buff.append(grey_scaled_observation)
          mini_batch = Agent.sample_replay_buffer(self.batch_size)

          if not done:
            rewardVal += (gamma * reward) 

          Agent.SGD(rewardVal, mini_batch)
          epsilon_decay_rate = max((e - time_step) / e, 0)
          epsilon -= epsilon_decay_rate
          
    if e % 100 == 0 and e > 0:
      self.test_env.render()    

  def main(self):
    dqn()



In [ ]:
if __name__ == "__main__":
  torch.device("cuda" if torch.cuda.is_available() else "cpu")
  DQN.main()
